# Tabular Data

Tabular data are data in a table format, such as an Excel spreadsheet.
We often need to work with tabular data.
*Pandas* is a powerful Python library for working with tabular data.

In this part, we will work with real-world data from the European Court of Human Rights [OpenData project](https://echr-opendata.eu/) (ECHR-OD).
ECHR-OD has data sets with cases from the European Court of Human Rights in different formats.
Here, we will work with data from the tabular data file `echr_2_0_0_structured_cases.csv`, which is in *CSV* format.
CSV is an abbreviation for "comma separated values".

You can get the 
[smaller subset](https://www.uio.no/studier/emner/jus/jus/JUS5080/h22/datasets/cases-100.csv)
from the semester page or the 
[full data set](https://echr-opendata.eu/download/2.0.0/structured/cases/csv) from ECHR-OD.


## pandas
We must first import the pandas library.

In [ ]:
import pandas as pd

```{note}
By convention, pandas is usually imported as `pd`.
```

Alternatively, we can import the functions we want to use.

In [ ]:
from pandas import Series, DataFrame, read_csv

Pandas has two different data types: `Series` and `DataFrame`.

## DataFrame
A pandas `DataFrame` is a table.
Tables are made up of rows and columns of data, we say they are two-dimensional.
A `DataFrame` can store numbers and strings as well as other types of data, for example dates.
We can make a `DataFrame` like this table with made-up data about some fictitious cases:

In [ ]:
table = DataFrame([[2, 3, 3],
                   [5, 4, 0],
                   [1, 2, 1]]
                  )
display(table)

By default, the rows are numbered.
We can add an index to give the rows names instead.


In [ ]:
table.index = ['Smith v. Jones',
               'Doe v. Doe',
               'Jones v. Doe']
display(table)

We can also give names to the columns.

In [ ]:
table.columns = ['time_allotted', 'time_used', 'witnesses']
display(table)

#hidden with "remove-cell"
import matplotlib.pyplot as plot
table.plot()
plot.savefig('temp/figure.svg')

## Reading CSV Files
Pandas has functions for importing data in many different formats.
Here, we will use CSV files.
To read the data set, we use pandas `read_csv()` function.

In [ ]:
cases = read_csv('cases-100.csv')

## Examining Data
Now, we can examine the data set.
Pandas has a method `head()` which prints the first five rows of the table.

In [ ]:
display(cases.head())

```{admonition} Display Function
:class: tip
We can use the Jupyter Notebook function `display()` to show a nicer format than with `print()`.
```

This is a large table with 374 columns. Therefore, pandas shows only the first and last columns.
We can look at the column names.

There are several interesting columns.
We will look at the conclusion columns in the section {ref}`relational_operators`.
Here are just a few examples:

| Column | Content |
| :---------- | :------ |
| `docname` | The name of the case. |
| `country.name` | The name of the country involved in the case. |
| `article=*` | Gives the relevant articles for each case. For example, the column `article=3` states whether Article 3 is mentioned. The value is 1 for `True` or 0 for `False`. |
| `ccl_article=*` | Conclusion for each article. For article 3, the conclusion is in the column `ccl_article=3` |

We can list all the column names like this:

In [ ]:
display(list(cases.columns))

## Specifying Index Column
When reading the CSV file, we can specify which column to use as index.
For the ECHR-OD data it makes sense to use `docname` as index.


In [ ]:
cases = read_csv('cases-100.csv', index_col='docname')
display(cases.head())

## Getting a Row
We can use the *locator* `.loc[]` to select parts of the table.
First, we select a row using the index.

In [ ]:
selection = cases.loc['CASE OF SKLYAR v. RUSSIA']
display(selection)

```{note}
Pandas only shows the first and last five columns.
To show all the data, we can convert them to a `list()`.
```

In [ ]:
print(list(selection))

## Listing a Column

We can use the subscript operator to get a column.

In [ ]:
selection = cases['country.name']
display(selection)

We can also use the locator `.loc[]` for this, then we need to supply two indexes.
If we want to get the entire `country.name` column, we leave out the indexes and just use a colon.

In [ ]:
selection = cases.loc[:, 'country.name']
display(selection)

```{note}
We have seen the colon before with lists.
With lists we can also leave out one or both of the indexes.
Without any indexes, we get a copy of the entire list:
`list_copy = old_list[:]`
```

## Using Numeric Indexes
Since we opened the table with the argument `index_col='docname'`, it has string indexes like *'CASE OF SKLYAR v. RUSSIA'*. But we can also use numeric indexes with the *integer locator* `.iloc[]`.
Here we get the first five rows using a range of indexes.

In [ ]:
selection = cases.iloc[0:4]
display(selection)

## Getting a Single Cell
We can get a single cell by specifying both the row and column names.
We can get the title in the row 'CASE OF SKLYAR v. RUSSIA'.

In [ ]:
selection = cases.loc['CASE OF SKLYAR v. RUSSIA', 'judgementdate']
print(selection)

```{note}
Tables are two-dimensional; therefore, we must use two indexes.
```

## Getting Multiple Columns
We can get multiple columns by specifying their names in a list.
Here, we use the subscript operator, not the locator `.loc[]`.


In [ ]:
selection = cases[['country.name', 'judgementdate']]
display(selection)

```{admonition} Missing Data
Here, some of the dates are missing.
Pandas uses `NaN` to indicate missing numeric data, short for *Not a Number*.
```

## Matching Multiple Columns
We can also select multiple columns using string matching on the column names.
The matching uses regular expressions, which we will discuss in {ref}`regex`.


In [ ]:
selection = cases.filter(regex='date', axis='columns')
display(selection)

(relational_operators)=
## Relational Operators
Pandas can select data conditionally, using relational operators.
The ECHR-OD case data has a column `ccl_article=2`.
This column has the value 1 for cases where there was found a violation of Article 2 of the European Convention on Human Rights.

We can use this column in an expression with a relational operator.

In [ ]:
cases['ccl_article=2'] == 1

```{admonition} Boolean Columns
The result is a Boolean column.
When we use them with pandas tables, relational operators return Boolean columns.
This is just like regular relational operators, except that regular relational operators return a single value.
```

## Relational Operators in Locators
We can use the result of a relational operator as a filter when selecting cases.
This will select the cases where the Boolean value is `True`.

In [ ]:
selection = cases.loc[cases['ccl_article=2'] == 1]
display(selection)

::::::{admonition} Conclusion Columns
The conclusions of each case are listed in the columns starting with `ccl_article`.
For Article N, the column name is `ccl_article=N`.
These columns have 3 possible values:

:::{table} Conclusion Values
:name: conclusion-values

| Value | Meaning |
|------:|:--------|
| 1 | violation |
| -1 | no violation |
| 0 | no conclusion / not relevant |
:::
::::::

Likewise, we can select cases where there was found no violation of Article 5 § 3.

In [ ]:
selection = cases.loc[cases['ccl_article=5-3'] == -1]
display(selection)

## Finding Text
We can't use a relational operators with text strings. Instead, we use `.str.contains()`:

In [ ]:
cases.loc[cases['country.name'].str.contains('Germany')]

## Combining Conditions
With pandas, we can't use `and` to combine expressions, instead we must use `&`:

In [ ]:
cases.loc[cases['country.name'].str.contains('Germany') & (cases['importance'] >= 3)]

:::{important}
Remember to use parentheses around the conditions with `&`. Otherwise, you will get unexpected results.
:::

## The isin() Method
With basic Python data types, we can check if they occur in a list with the operator `in`, for example `5 in [0, 1]` is `False`.
Likewise, pandas has the method `.isin()` for checking if table values occur in a list.
We can use this for example for finding cases where the conclusion is "violation" or "no violation", filtering out cases where an article is not relevant.

As the table {ref}`conclusion-values` shows, the value is 1 for "violation" or -1 for "no violation".
Again, we try with Article 5 § 3.

In [ ]:
selection = cases.loc[cases['ccl_article=5-3'].isin([-1, 1])]
display(selection)

## Selecting by Year
We can select cases by the decision date.
First, we must convert the `decisiondate` column to machine-readable format.

In [ ]:
cases['decisiondate'] = pd.to_datetime(cases['decisiondate'], dayfirst = True)

```{tip}
We specify the date format with the argument `dayfirst = True`.
There is also an argument `yearfirst`.
```

```{note}
Python has the built-in library `datetime` for handling time and dates.
But when we use pandas, we usually only need pandas' `to_datetime()` function.
```

Now, we can use the year from the decision date with a relational operator.

In [ ]:
selection = cases.loc[cases['decisiondate'].dt.year == 2005]
display(selection)

## Aggregating Data
Pandas has various functions for aggregating data.
For example, we can count the number of occurrences of different values with `value_counts()`.

The column `doctypebranch` contains the type of deciding body for the case.

In [ ]:
selection = cases['doctypebranch'].value_counts()
print(selection)

## Visualizing Data
We can also plot our results.
Here we continue working with our selection.

In [ ]:
selection.plot(kind='bar')

## Sums
Pandas has a function for calculating the sum of columns or rows.
Since our data uses the number `1` to indicate `True`, we can count matching cases simply by summing the ones.

We can find the number of cases where Article 3 applies by summing the column `article=3`.

In [ ]:
result = cases.loc[:, 'article=3'].sum()
print(result)

We can also sum the values of multiple columns at once.

In [ ]:
result = cases.loc[:, ['article=3', 'article=5', 'article=6', 'article=10']].sum()

And plot the result.

In [ ]:
result.plot(kind='bar')

## Choosing Axis
As we have seen, the method `.sum()`without any parameters will calculate the sum downwards, along the *index* of the table.
We are adding together the rows of the table.

In [ ]:
result = cases.loc[:, ['article=3', 'article=5', 'article=6', 'article=10']].sum()
display(result)

We get the same result if we specify the parameter `axis = 'index'`.
This is because `'index'` is the default value for the parameter `axis`.

In [ ]:
result = cases.loc[:, ['article=3', 'article=5', 'article=6', 'article=10']].sum(axis='index')
display(result)

We can also add together the columns of the table.
To do this, we must change the direction to `axis = 'column'`.

In [ ]:
result = cases.loc[:, ['article=3', 'article=5', 'article=6', 'article=10']].sum(axis='columns')
display(result)

The result is a column with 100 items. These are the number of relevant articles
(of the four we selected) for the 100 cases.

## Series
A pandas `Series` is a list of values, similar to a Python list.
When we selected a single row or column above, the result was a `Series`.

We can select a single column from our small example `DataFrame`.

In [ ]:
column = table['time_used']
print(column)

We can select a row with `.loc[ ]`.

In [ ]:
row = table.loc['Doe v. Doe']
print(row)

If we need to, we can also make a `Series` from scratch:

In [ ]:
row = Series([3, 1, 5])
print(row)